In [1]:
import pandas as pd
import numpy as np

In [2]:
# Lecture du fichier *.txt
data = pd.read_csv('fredmq.txt', sep=',').set_index(keys='Date')
# Récupération des types de transformations dans un dictionnaire
transformations = dict(data.loc['Transform']) 
# Abandon de la ligne des transformations
data.drop(labels='Transform', inplace=True)
# Ajustement de l'indice du DataFrame au format datetime
data.index = pd.to_datetime(arg=data.index)
# Abandon des colonnes ayant un nombre de NaN's >= 30
data.dropna(thresh=len(data) - 30, axis=1, inplace=True)
# Abandon des NaN's restants
data.dropna(axis=0, inplace=True)

In [3]:
# Création du DataFrame stockant les séries transformées
transformed_data = pd.DataFrame()
# Transformation des séries selon les codes fournis par McCracken et Ng
for col in data.columns:
    # Récupération du code de transformation
    code = transformations[col]
    # Récupération de la série individuelle
    subset = data[col]
    # Application des transformées selon le code correspondant
    if code == 1:
        # Aucune transformation à appliquer
        temp = subset
    elif code == 2:
        # Différence première
        temp = subset.diff(periods=1)
    elif code == 3:
        # Différence seconde 
        temp = subset.diff(periods=1).diff(periods=1)
    elif code == 4:
        # Transformée en log
        temp = np.log(subset)
    elif code == 5:
        # Différence première du log
        temp = np.log(subset).diff(periods=1)
    elif code == 6:
        # Différence seconde du log 
        temp = np.log(subset).diff(periods=1).diff(periods=1)
    elif code == 7:
        # Différence première de la variation relative
        temp = subset.pct_change().diff(periods=1)
    # Aggrégation des séries transformées
    transformed_data = pd.concat([transformed_data, temp], axis=1)
# Abandon des NaN's causées par les différences premières et secondes
transformed_data.dropna(axis=0, inplace=True)

In [4]:
# Calcul de la médiane des séries
medians = transformed_data.median(axis=0)
# DataFrame contenant les médianes des séries en chaque point
mdf =  transformed_data * 0 + medians
# Calcul de la distance entre les observations et les médianes
z = abs(transformed_data - mdf)
# Calcul de la gamme interquartile des séries
irq = transformed_data.quantile(q=.75) - transformed_data.quantile(q=.25)
# DataFrame contenant les gammes interquartiles des séries en chaque point
irqdf = transformed_data * 0 + irq
# Détermination des outliers (x est un outlier si abs(x - mediane) > 10 * gamme interquartile)
outliers = z > 10 * irqdf
# Abandon des observations considérées comme outliers
mapping = transformed_data[outliers == False].dropna(axis=0)

In [6]:
# Sélection de la période d'estimation
estimation_sample = mapping.loc[(mapping.index >= pd.to_datetime('1983-01-01')) 
                                & (mapping.index <= pd.to_datetime('2016-12-01'))]
# Sélection de la période de prévision
forecast_sample = mapping.loc[mapping.index >= pd.to_datetime('2017-01-01')]
# Standardisation des données d'estimation
estimation_sample_std = (estimation_sample - estimation_sample.mean(axis=0)) / estimation_sample.std(axis=0)